In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast

class Transformer():
    def __init__(self):
        pass
    def transform(self, inputDFs):
        pass

class FirstTransformer(Transformer):
    def transform(self,inputDFs):
        """  
        Customer who have brought Airpods after buying the iPhone
        """
        transactionInputDF = inputDFs.get("transactionInputDF")
        print("transactionInputDF in transform")

        transactionInputDF.show()

        window_spec = Window.partitionBy("customer_id").orderBy("transaction_date")

        transformedDF = transactionInputDF.withColumn("next_product_name", lead("product_name").over(window_spec))

        print("Airpods after buyinh iPhone")
        transformedDF.orderBy("customer_id","transaction_date","product_name").show()

        filteredDF = transformedDF.filter(
            (col("product_name") == "iPhone") & (col("next_product_name") == "AirPods")
        )
        filteredDF.orderBy("customer_id","transaction_date","product_name").show()

        customerInputDF = inputDFs.get("customerInputDF")

        joined_table = customerInputDF.join(\
            broadcast(filteredDF)\
            ,on="customer_id")
        
        print("joined table")
        joined_table.show()

        return joined_table\
        .select("customer_id","customer_name","location")\
        .show()


